In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import cv2
import numpy as np
from app.utils.ResNet34Model import ResNet34Model
from app.utils.HistogramEqualization import HistogramEqualization

In [ ]:
%run DatasetStatistics.ipynb

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
checkpoint = torch.load(
    "./config/resnet34.pth", map_location=device, weights_only=False
)

In [ ]:
print(checkpoint.keys())

In [ ]:
IMAGE_SIZE = checkpoint["image_size"]
MEAN = checkpoint["mean"]
STD = checkpoint["std"]
DIR = "./new_data/Testing"

In [ ]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        HistogramEqualization(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[MEAN] * 3, std=[STD] * 3),
    ]
)

In [ ]:
dataset = datasets.ImageFolder(DIR, transform=transforms)
print(dataset.classes)

In [ ]:
dl = DataLoader(dataset, shuffle=False, batch_size=1)

In [ ]:
model = ResNet34Model()
model.load_state_dict(checkpoint["weights"])
model.to(device)
model.eval()

In [ ]:
img, _ = next(iter(dl))
img = img.to(device)

In [ ]:
out = model(img)

In [ ]:
class_index = out.argmax(dim=1).item()

In [ ]:
model.zero_grad()
loss = out[:, class_index].backward()

In [ ]:
grads = model.get_activations_gradient()
pool_gradients = torch.mean(grads, dim=[0, 2, 3])

In [ ]:
activations = model.get_activations().clone()

for i in range(512):
    activations[:, i, :, :] *= pool_gradients[i]

In [ ]:
heatmap = torch.mean(activations, dim=1).squeeze()
heatmap = F.relu(heatmap)  # reLU

In [ ]:
heatmap /= heatmap.max()  # normalize

In [ ]:
heatmap = heatmap.cpu().numpy()
plt.matshow(heatmap.squeeze())

In [ ]:
img_path, _ = dataset.samples[0]
img_original = cv2.imread(img_path)
img_original = cv2.resize(img_original, IMAGE_SIZE)

In [ ]:
heatmap = cv2.resize(heatmap, (img_original.shape[1], img_original.shape[0]))
heatmap = (heatmap * 255).astype(np.uint8)

In [ ]:
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

In [ ]:
superimposed_img = cv2.addWeighted(img_original, 0.6, heatmap, 0.4, 0)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 5))

img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
axs[0].imshow(img_original)
axs[0].set_title(f"Original image ({dataset.classes[_]} class)")
axs[0].axis("off")

axs[1].imshow(cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB), cmap="jet")
axs[1].set_title("Grad-CAM")
axs[1].axis("off")

superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
axs[2].imshow(superimposed_img, cmap="jet")
axs[2].set_title(
    f"Grad-CAM over the original image\n(predicted {dataset.classes[class_index]} class)"
)
axs[2].axis("off")

plt.tight_layout()
plt.show()